In [70]:
import re, subprocess, pyodbc , json
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd
from faker import Faker
fake = Faker()

In [5]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode, QuestionTypeID, QTEXT    
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%2022-Q1-CIO%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()


In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT QPK, QGROUP, IdText, QuestionTypeCode , QuestionTypeCode, QTEXT, help_text   
FROM  vw_MetricsCompositeKeys
WHERE PK_Form LIKE '%2022-Q1-CIO%'
"""  
df = pd.read_sql(query,con=conn) 
conn.close()

In [6]:
offset = 10
qg = 3200
fp = 3000

In [8]:
sections = [ 
{'id':'7a', 'text': 'Ground Truth Testing- Testing Activities'},
{'id':'7b', 'text': 'Ground Truth Testing- Penetration Testing'},
{'id':'7c', 'text': 'Ground Truth Testing- Vulnerability Disclosure Tools'},
{'id':'7d', 'text': 'Ground Truth Testing- Team Exercises'},
{'id':'8', 'text': 'Smart Patching'},
{'id':'9', 'text': 'Vulnerability Disclosure'},
{'id':'10', 'text': 'Resilience'}  
]
df_s = pd.DataFrame(sections)
df_s

,id,text
0,7a,Ground Truth Testing- Testing Activities
1,7b,Ground Truth Testing- Penetration Testing
2,7c,Ground Truth Testing- Vulnerability Disclosure...
3,7d,Ground Truth Testing- Team Exercises
4,8,Smart Patching
5,9,Vulnerability Disclosure
6,10,Resilience


In [71]:
for i,r in df_s.iterrows():
    n = offset+i
    p = f"(@PK_FormPage+{n},@PK_Form, '{r.id}: {r.text}', 'FismaForms/2022/2022_Q1_CIO_{r['id']}.aspx', {n}), ";
    print( p ) 

(@PK_FormPage+9,@PK_Form, '7a: Ground Truth Testing- Testing Activities', 'FismaForms/2022/2022_Q1_CIO_7a.aspx', 9), 
(@PK_FormPage+10,@PK_Form, '7b: Ground Truth Testing- Penetration Testing', 'FismaForms/2022/2022_Q1_CIO_7b.aspx', 10), 
(@PK_FormPage+11,@PK_Form, '7c: Ground Truth Testing- Vulnerability Disclosure Tools', 'FismaForms/2022/2022_Q1_CIO_7c.aspx', 11), 
(@PK_FormPage+12,@PK_Form, '7d: Ground Truth Testing- Team Exercises', 'FismaForms/2022/2022_Q1_CIO_7d.aspx', 12), 
(@PK_FormPage+13,@PK_Form, '8: Smart Patching', 'FismaForms/2022/2022_Q1_CIO_8.aspx', 13), 
(@PK_FormPage+14,@PK_Form, '9: Vulnerability Disclosure', 'FismaForms/2022/2022_Q1_CIO_9.aspx', 14), 
(@PK_FormPage+15,@PK_Form, '10: Resilience', 'FismaForms/2022/2022_Q1_CIO_10.aspx', 15), 


In [72]:
for i,r in df_s.iterrows():
    n = offset+i
    p = f"(@PK_QGroup+{n},'{r.id}','Section {n}: Identify - {r.text}',@PK_Form, @PK_FormPage+{n}, {n}),";
    print( p ) 

(@PK_QGroup+9,'7a','Section 9: Identify - Ground Truth Testing- Testing Activities',@PK_Form, @PK_FormPage+9, 9),
(@PK_QGroup+10,'7b','Section 10: Identify - Ground Truth Testing- Penetration Testing',@PK_Form, @PK_FormPage+10, 10),
(@PK_QGroup+11,'7c','Section 11: Identify - Ground Truth Testing- Vulnerability Disclosure Tools',@PK_Form, @PK_FormPage+11, 11),
(@PK_QGroup+12,'7d','Section 12: Identify - Ground Truth Testing- Team Exercises',@PK_Form, @PK_FormPage+12, 12),
(@PK_QGroup+13,'8','Section 13: Identify - Smart Patching',@PK_Form, @PK_FormPage+13, 13),
(@PK_QGroup+14,'9','Section 14: Identify - Vulnerability Disclosure',@PK_Form, @PK_FormPage+14, 14),
(@PK_QGroup+15,'10','Section 15: Identify - Resilience',@PK_Form, @PK_FormPage+15, 15),


In [73]:
def cleaner(s):
    try:
        s=s.replace('\'','"')
        s=s.replace('�','')
        s=s.replace('nan','')
        s=re.sub('[\u0080-\uFFFF]+', '', s)
    except:
        pass
    return s

In [74]:
l = [] 
s=10
qpk = 27200
for i, r in df_s[1:].iterrows():  
    dff = pd.read_csv(fr'C:\Users\Tim\Documents\CIO\{r.id}.csv')
    dff.fillna('') 
    dff = dff.astype(str)
    for ii, rr in dff.iterrows():  
        qtext = cleaner(rr['qtext'])
        qtype = cleaner(str(rr['qtype']))
        if pd.isna(qtype):
            qtype = '0'
        qtype = re.sub(r"[^\d]", "", qtype)
        qid = re.sub(r"[^\d{1,2}\.]", "", rr['id'])
        if len(qtext) > 10: 
            print( f"({qpk}, @FormName, @PK_QGroup+{s},{ii+1},N\'{qid}\', {qtype}, N\'{qtext}\', NULL)," )
            qpk=qpk+1
    s=s+1
    qpk=qpk+25
 
 

(27200, @FormName, @PK_QGroup+10,1,N'7.2', , N'For the following questions, provide responses for the last reporting period', NULL),
(27201, @FormName, @PK_QGroup+10,2,N'7.2.1', 20, N'How many government FTEs directly performed penetration testing on agency information systems for your organization?', NULL),
(27202, @FormName, @PK_QGroup+10,3,N'7.2.2', 20, N'How many contractor FTEs directly performed penetration testing on agency information systems?', NULL),
(27203, @FormName, @PK_QGroup+10,4,N'7.2.3', 20, N'How many penetration tests (from 7.1) were conducted via an external contract?', NULL),
(27204, @FormName, @PK_QGroup+10,5,N'7.2.4', 350, N'What was the cost (in millions of dollars) of penetration tests (from 7.1) conducted?', NULL),
(27205, @FormName, @PK_QGroup+10,6,N'7.2.5', 90, N'What tools were used to perform penetration tests?', NULL),
(27206, @FormName, @PK_QGroup+10,7,N'7.2.6', 90, N'What tools were used to perform static and dynamic code analysis?', NULL),
(27232, @For